# Módulo 10: Atividade Final Prática## Objetivos de Aprendizagem- Aplicar todos os conceitos aprendidos em um projeto completo- Implementar uma solução de visão computacional do zero- Integrar diferentes técnicas e abordagens- Desenvolver habilidades práticas de implementação- Apresentar resultados e análises técnicas---## 10.1 Projeto Final: Sistema de Análise de Imagens Multimodal### Descrição do Projeto**Objetivo:** Desenvolver um sistema completo de análise de imagens que integre diferentes técnicas de visão computacional aprendidas ao longo do curso.![Descrição Projeto Final](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo10/descricao_projeto_final.png)**Funcionalidades Principais:**1. **Classificação de Imagens** usando CNNs tradicionais2. **Detecção de Objetos** com YOLO simplificado3. **Segmentação de Imagens** usando U-Net4. **OCR e Extração de Texto** com Tesseract/EasyOCR5. **Análise Multimodal** com Foundation Models (CLIP/GPT-4V)6. **Interface de Usuário** para interação### Arquitetura do Sistema**Estrutura Modular:**![Arquitetura Sistema](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo10/arquitetura_sistema.png)**Componentes:**- **Input Module**: Entrada de imagens- **Preprocessing Module**: Pré-processamento- **Analysis Module**: Análise multimodal- **Output Module**: Resultados e visualização### Tecnologias Utilizadas**Frameworks:**- **PyTorch**: Deep learning- **OpenCV**: Processamento de imagem- **Transformers**: Modelos pré-treinados- **Streamlit**: Interface web**Modelos:**- **ResNet**: Classificação- **YOLO**: Detecção- **U-Net**: Segmentação- **CLIP**: Análise multimodal![Tecnologias Utilizadas](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo10/tecnologias_utilizadas.png)**Referências:**- [Deep Residual Learning for Image Recognition - He et al.](https://arxiv.org/abs/1512.03385)- [You Only Look Once: Unified, Real-Time Object Detection - Redmon et al.](https://arxiv.org/abs/1506.02640)

## 10.2 Demonstração Prática: Sistema Completo

Vamos implementar e demonstrar o sistema completo:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional
import json
from datetime import datetime

class MultimodalImageAnalysisSystem:
    """Sistema completo de análise de imagens multimodal"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.results = {}
        
        # Inicializar módulos
        self.classifier = self._build_classifier()
        self.detector = self._build_detector()
        self.segmenter = self._build_segmenter()
        self.ocr_engine = self._build_ocr_engine()
        self.multimodal_analyzer = self._build_multimodal_analyzer()
        
        # Transformações
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def _build_classifier(self):
        """Constrói o classificador de imagens"""
        
        class SimpleCNN(nn.Module):
            def __init__(self, num_classes=10):
                super(SimpleCNN, self).__init__()
                
                self.features = nn.Sequential(
                    nn.Conv2d(3, 32, 3, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2),
                    nn.Conv2d(32, 64, 3, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2),
                    nn.Conv2d(64, 128, 3, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2),
                    nn.Conv2d(128, 256, 3, padding=1),
                    nn.ReLU(),
                    nn.AdaptiveAvgPool2d((1, 1))
                )
                
                self.classifier = nn.Sequential(
                    nn.Dropout(0.5),
                    nn.Linear(256, 128),
                    nn.ReLU(),
                    nn.Dropout(0.5),
                    nn.Linear(128, num_classes)
                )
                
            def forward(self, x):
                x = self.features(x)
                x = x.view(x.size(0), -1)
                x = self.classifier(x)
                return x
        
        return SimpleCNN(num_classes=10).to(self.device)
    
    def _build_detector(self):
        """Constrói o detector de objetos"""
        
        class SimpleYOLO(nn.Module):
            def __init__(self, num_classes=5):
                super(SimpleYOLO, self).__init__()
                
                self.backbone = nn.Sequential(
                    nn.Conv2d(3, 32, 3, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2),
                    nn.Conv2d(32, 64, 3, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2),
                    nn.Conv2d(64, 128, 3, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2)
                )
                
                # Output: [batch, 5 + num_classes, H, W]
                # 5 = [x, y, w, h, confidence]
                self.detection_head = nn.Conv2d(128, 5 + num_classes, 1)
                
            def forward(self, x):
                features = self.backbone(x)
                detections = self.detection_head(features)
                return detections
        
        return SimpleYOLO(num_classes=5).to(self.device)
    
    def _build_segmenter(self):
        """Constrói o segmentador de imagens"""
        
        class SimpleUNet(nn.Module):
            def __init__(self, num_classes=3):
                super(SimpleUNet, self).__init__()
                
                # Encoder
                self.enc1 = nn.Sequential(
                    nn.Conv2d(3, 64, 3, padding=1),
                    nn.ReLU(),
                    nn.Conv2d(64, 64, 3, padding=1),
                    nn.ReLU()
                )
                
                self.enc2 = nn.Sequential(
                    nn.MaxPool2d(2),
                    nn.Conv2d(64, 128, 3, padding=1),
                    nn.ReLU(),
                    nn.Conv2d(128, 128, 3, padding=1),
                    nn.ReLU()
                )
                
                # Decoder
                self.dec1 = nn.Sequential(
                    nn.ConvTranspose2d(128, 64, 2, stride=2),
                    nn.Conv2d(128, 64, 3, padding=1),
                    nn.ReLU(),
                    nn.Conv2d(64, 64, 3, padding=1),
                    nn.ReLU()
                )
                
                self.final = nn.Conv2d(64, num_classes, 1)
                
            def forward(self, x):
                # Encoder
                enc1 = self.enc1(x)
                enc2 = self.enc2(enc1)
                
                # Decoder
                dec1 = self.dec1(enc2)
                
                # Skip connection
                dec1 = torch.cat([dec1, enc1], dim=1)
                
                # Final output
                output = self.final(dec1)
                
                return output
        
        return SimpleUNet(num_classes=3).to(self.device)
    
    def _build_ocr_engine(self):
        """Constrói o motor OCR"""
        
        class SimpleOCR:
            def __init__(self):
                self.text_regions = []
                
            def preprocess_image(self, image):
                """Pré-processa imagem para OCR"""
                # Converter para escala de cinza
                if len(image.shape) == 3:
                    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                else:
                    gray = image
                
                # Aplicar threshold
                _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                
                # Reduzir ruído
                kernel = np.ones((3, 3), np.uint8)
                cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
                
                return cleaned
            
            def detect_text_regions(self, image):
                """Detecta regiões de texto"""
                # Usar contours para detectar regiões
                contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                text_regions = []
                for contour in contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    if w > 20 and h > 10:  # Filtrar regiões muito pequenas
                        text_regions.append((x, y, w, h))
                
                return text_regions
            
            def extract_text(self, image):
                """Extrai texto da imagem"""
                # Simular extração de texto
                # Em implementação real, usar Tesseract ou EasyOCR
                
                preprocessed = self.preprocess_image(image)
                text_regions = self.detect_text_regions(preprocessed)
                
                # Simular texto extraído
                extracted_text = f"Texto detectado em {len(text_regions)} regiões"
                
                return extracted_text, text_regions
        
        return SimpleOCR()
    
    def _build_multimodal_analyzer(self):
        """Constrói o analisador multimodal"""
        
        class MultimodalAnalyzer:
            def __init__(self):
                self.text_encoder = nn.Sequential(
                    nn.Linear(100, 128),
                    nn.ReLU(),
                    nn.Linear(128, 128)
                )
                
                self.image_encoder = nn.Sequential(
                    nn.Linear(224*224*3, 512),
                    nn.ReLU(),
                    nn.Linear(512, 128)
                )
                
            def analyze_multimodal(self, image, text):
                """Analisa conteúdo multimodal"""
                # Simular análise multimodal
                
                # Análise de imagem
                image_features = self.image_encoder(image.flatten())
                
                # Análise de texto
                text_features = self.text_encoder(torch.randn(100))
                
                # Similaridade
                similarity = F.cosine_similarity(image_features, text_features, dim=0)
                
                # Análise semântica
                analysis = {
                    'similarity': similarity.item(),
                    'image_objects': ['objeto1', 'objeto2', 'objeto3'],
                    'text_entities': ['entidade1', 'entidade2'],
                    'semantic_match': similarity.item() > 0.5
                }
                
                return analysis
        
        return MultimodalAnalyzer().to(self.device)
    
    def create_sample_image(self, width=224, height=224):
        """Cria uma imagem de exemplo para demonstração"""
        
        # Criar imagem base
        image = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Adicionar formas geométricas
        # Círculo vermelho
        cv2.circle(image, (80, 80), 30, (255, 0, 0), -1)
        
        # Retângulo azul
        cv2.rectangle(image, (150, 50), (200, 100), (0, 255, 0), -1)
        
        # Triângulo amarelo
        pts = np.array([[100, 150], [80, 200], [120, 200]], np.int32)
        cv2.fillPoly(image, [pts], (0, 0, 255))
        
        # Adicionar texto simulado
        cv2.putText(image, 'SAMPLE', (50, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return image
    
    def analyze_image(self, image):
        """Analisa uma imagem usando todos os módulos"""
        
        results = {
            'timestamp': datetime.now().isoformat(),
            'image_shape': image.shape,
            'analysis': {}
        }
        
        # 1. Classificação
        print("Executando classificação...")
        classification_result = self._classify_image(image)
        results['analysis']['classification'] = classification_result
        
        # 2. Detecção de objetos
        print("Executando detecção de objetos...")
        detection_result = self._detect_objects(image)
        results['analysis']['detection'] = detection_result
        
        # 3. Segmentação
        print("Executando segmentação...")
        segmentation_result = self._segment_image(image)
        results['analysis']['segmentation'] = segmentation_result
        
        # 4. OCR
        print("Executando OCR...")
        ocr_result = self._extract_text(image)
        results['analysis']['ocr'] = ocr_result
        
        # 5. Análise multimodal
        print("Executando análise multimodal...")
        multimodal_result = self._analyze_multimodal(image, ocr_result['text'])
        results['analysis']['multimodal'] = multimodal_result
        
        self.results = results
        return results
    
    def _classify_image(self, image):
        """Classifica a imagem"""
        # Converter para tensor
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        
        # Classificar
        self.classifier.eval()
        with torch.no_grad():
            outputs = self.classifier(image_tensor)
            probabilities = F.softmax(outputs, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0, predicted_class].item()
        
        return {
            'predicted_class': predicted_class,
            'confidence': confidence,
            'probabilities': probabilities[0].cpu().numpy().tolist()
        }
    
    def _detect_objects(self, image):
        """Detecta objetos na imagem"""
        # Converter para tensor
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        
        # Detectar
        self.detector.eval()
        with torch.no_grad():
            detections = self.detector(image_tensor)
        
        # Simular detecções
        objects = [
            {'class': 'circle', 'confidence': 0.95, 'bbox': [50, 50, 60, 60]},
            {'class': 'rectangle', 'confidence': 0.87, 'bbox': [140, 40, 60, 60]},
            {'class': 'triangle', 'confidence': 0.92, 'bbox': [80, 140, 40, 60]}
        ]
        
        return {
            'num_objects': len(objects),
            'objects': objects
        }
    
    def _segment_image(self, image):
        """Segmenta a imagem"""
        # Converter para tensor
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        
        # Segmentar
        self.segmenter.eval()
        with torch.no_grad():
            segmentation = self.segmenter(image_tensor)
            segmentation = F.softmax(segmentation, dim=1)
            predicted_segments = torch.argmax(segmentation, dim=1)
        
        return {
            'num_segments': 3,
            'segmentation_map': predicted_segments[0].cpu().numpy().tolist()
        }
    
    def _extract_text(self, image):
        """Extrai texto da imagem"""
        text, regions = self.ocr_engine.extract_text(image)
        
        return {
            'text': text,
            'num_regions': len(regions),
            'regions': regions
        }
    
    def _analyze_multimodal(self, image, text):
        """Analisa conteúdo multimodal"""
        # Converter imagem para tensor
        image_tensor = torch.FloatTensor(image).flatten().to(self.device)
        
        # Análise multimodal
        analysis = self.multimodal_analyzer.analyze_multimodal(image_tensor, text)
        
        return analysis
    
    def visualize_results(self, image, results):
        """Visualiza os resultados da análise"""
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # Imagem original
        axes[0, 0].imshow(image)
        axes[0, 0].set_title('Imagem Original')
        axes[0, 0].axis('off')
        
        # Classificação
        classification = results['analysis']['classification']
        axes[0, 1].bar(range(len(classification['probabilities'])), classification['probabilities'])
        axes[0, 1].set_title(f'Classificação\nClasse: {classification["predicted_class"]}, Conf: {classification["confidence"]:.2f}')
        axes[0, 1].set_xlabel('Classe')
        axes[0, 1].set_ylabel('Probabilidade')
        
        # Detecção de objetos
        detection = results['analysis']['detection']
        detection_image = image.copy()
        for obj in detection['objects']:
            x, y, w, h = obj['bbox']
            cv2.rectangle(detection_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(detection_image, f"{obj['class']}: {obj['confidence']:.2f}", 
                        (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        axes[0, 2].imshow(detection_image)
        axes[0, 2].set_title(f'Detecção de Objetos\n{detection["num_objects"]} objetos detectados')
        axes[0, 2].axis('off')
        
        # Segmentação
        segmentation = results['analysis']['segmentation']
        seg_map = np.array(segmentation['segmentation_map'])
        axes[1, 0].imshow(seg_map, cmap='viridis')
        axes[1, 0].set_title(f'Segmentação\n{segmentation["num_segments"]} segmentos')
        axes[1, 0].axis('off')
        
        # OCR
        ocr = results['analysis']['ocr']
        ocr_image = image.copy()
        for region in ocr['regions']:
            x, y, w, h = region
            cv2.rectangle(ocr_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        axes[1, 1].imshow(ocr_image)
        axes[1, 1].set_title(f'OCR\n{ocr["num_regions"]} regiões de texto')
        axes[1, 1].axis('off')
        
        # Análise multimodal
        multimodal = results['analysis']['multimodal']
        axes[1, 2].text(0.1, 0.8, f'Similaridade: {multimodal["similarity"]:.3f}', transform=axes[1, 2].transAxes)
        axes[1, 2].text(0.1, 0.6, f'Match Semântico: {multimodal["semantic_match"]}', transform=axes[1, 2].transAxes)
        axes[1, 2].text(0.1, 0.4, f'Objetos: {len(multimodal["image_objects"])}', transform=axes[1, 2].transAxes)
        axes[1, 2].text(0.1, 0.2, f'Entidades: {len(multimodal["text_entities"])}', transform=axes[1, 2].transAxes)
        axes[1, 2].set_title('Análise Multimodal')
        axes[1, 2].axis('off')
        
        plt.suptitle('Sistema de Análise de Imagens Multimodal', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        return fig
    
    def generate_report(self, results):
        """Gera relatório da análise"""
        
        report = {
            'summary': {
                'timestamp': results['timestamp'],
                'image_shape': results['image_shape'],
                'total_analyses': len(results['analysis'])
            },
            'classification': {
                'predicted_class': results['analysis']['classification']['predicted_class'],
                'confidence': results['analysis']['classification']['confidence']
            },
            'detection': {
                'num_objects': results['analysis']['detection']['num_objects'],
                'objects': results['analysis']['detection']['objects']
            },
            'segmentation': {
                'num_segments': results['analysis']['segmentation']['num_segments']
            },
            'ocr': {
                'text': results['analysis']['ocr']['text'],
                'num_regions': results['analysis']['ocr']['num_regions']
            },
            'multimodal': {
                'similarity': results['analysis']['multimodal']['similarity'],
                'semantic_match': results['analysis']['multimodal']['semantic_match']
            }
        }
        
        return report

def demonstrate_complete_system():
    """Demonstra o sistema completo"""
    
    print("=== Sistema de Análise de Imagens Multimodal ===")
    print("\nNota: Esta demonstração usa modelos simplificados para fins educacionais.")
    print("Em aplicações reais, use modelos pré-treinados e otimizados.")
    
    # Criar instância do sistema
    system = MultimodalImageAnalysisSystem()
    
    # Criar imagem de exemplo
    print("\nCriando imagem de exemplo...")
    sample_image = system.create_sample_image()
    
    # Analisar imagem
    print("\nAnalisando imagem...")
    results = system.analyze_image(sample_image)
    
    # Visualizar resultados
    print("\nVisualizando resultados...")
    fig = system.visualize_results(sample_image, results)
    
    # Gerar relatório
    print("\nGerando relatório...")
    report = system.generate_report(results)
    
    # Exibir resumo
    print("\n=== RESUMO DA ANÁLISE ===")
    print(f"Timestamp: {report['summary']['timestamp']}")
    print(f"Formato da imagem: {report['summary']['image_shape']}")
    print(f"Análises realizadas: {report['summary']['total_analyses']}")
    print(f"\nClassificação: Classe {report['classification']['predicted_class']} (Conf: {report['classification']['confidence']:.2f})")
    print(f"Detecção: {report['detection']['num_objects']} objetos")
    print(f"Segmentação: {report['segmentation']['num_segments']} segmentos")
    print(f"OCR: {report['ocr']['num_regions']} regiões de texto")
    print(f"Multimodal: Similaridade {report['multimodal']['similarity']:.3f}")
    
    return system, sample_image, results, report

# Executar demonstração
complete_system, demo_image, analysis_results, final_report = demonstrate_complete_system()

### Análise dos Resultados

**Sistema Completo Observado:**

1. **Classificação**: CNN para classificação de imagens
2. **Detecção**: YOLO simplificado para detecção de objetos
3. **Segmentação**: U-Net para segmentação semântica
4. **OCR**: Extração de texto com detecção de regiões
5. **Multimodal**: Análise combinada de imagem e texto
6. **Visualização**: Resultados integrados em dashboard

**Insights Importantes:**
- **Integração**: Múltiplas técnicas trabalhando juntas
- **Pipeline**: Fluxo completo de análise
- **Visualização**: Resultados claros e interpretáveis
- **Relatório**: Documentação completa da análise

**Referências:**
- [Deep Residual Learning for Image Recognition - He et al.](https://arxiv.org/abs/1512.03385)
- [You Only Look Once: Unified, Real-Time Object Detection - Redmon et al.](https://arxiv.org/abs/1506.02640)


## 10.3 Implementação de Interface Web

### Streamlit Dashboard

Vamos criar uma interface web para o sistema:


In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from io import BytesIO
import base64

class WebInterface:
    """Interface web para o sistema de análise"""
    
    def __init__(self):
        self.system = MultimodalImageAnalysisSystem()
        
    def create_dashboard(self):
        """Cria o dashboard principal"""
        
        st.set_page_config(
            page_title="Sistema de Análise de Imagens Multimodal",
            page_icon="🔍",
            layout="wide"
        )
        
        st.title("🔍 Sistema de Análise de Imagens Multimodal")
        st.markdown("---")
        
        # Sidebar
        st.sidebar.title("Configurações")
        
        # Upload de imagem
        uploaded_file = st.sidebar.file_uploader(
            "Upload de Imagem",
            type=['png', 'jpg', 'jpeg'],
            help="Faça upload de uma imagem para análise"
        )
        
        # Opções de análise
        st.sidebar.subheader("Módulos de Análise")
        enable_classification = st.sidebar.checkbox("Classificação", value=True)
        enable_detection = st.sidebar.checkbox("Detecção de Objetos", value=True)
        enable_segmentation = st.sidebar.checkbox("Segmentação", value=True)
        enable_ocr = st.sidebar.checkbox("OCR", value=True)
        enable_multimodal = st.sidebar.checkbox("Análise Multimodal", value=True)
        
        # Botão de análise
        if st.sidebar.button("Analisar Imagem", type="primary"):
            if uploaded_file is not None:
                self._analyze_uploaded_image(uploaded_file, {
                    'classification': enable_classification,
                    'detection': enable_detection,
                    'segmentation': enable_segmentation,
                    'ocr': enable_ocr,
                    'multimodal': enable_multimodal
                })
            else:
                st.error("Por favor, faça upload de uma imagem primeiro.")
        
        # Demo com imagem de exemplo
        if st.sidebar.button("Usar Imagem de Exemplo"):
            self._demo_with_sample_image()
        
        # Exibir imagem atual
        if 'current_image' in st.session_state:
            st.subheader("Imagem Atual")
            st.image(st.session_state['current_image'], caption="Imagem para análise", use_column_width=True)
        
        # Exibir resultados
        if 'analysis_results' in st.session_state:
            self._display_results(st.session_state['analysis_results'])
        
        # Estatísticas do sistema
        self._display_system_stats()
    
    def _analyze_uploaded_image(self, uploaded_file, options):
        """Analisa imagem enviada"""
        
        # Converter para numpy array
        image = np.array(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Redimensionar se necessário
        if image.shape[0] > 512 or image.shape[1] > 512:
            image = cv2.resize(image, (512, 512))
        
        # Armazenar imagem
        st.session_state['current_image'] = image
        
        # Analisar
        with st.spinner('Analisando imagem...'):
            results = self.system.analyze_image(image)
            st.session_state['analysis_results'] = results
        
        st.success("Análise concluída!")
    
    def _demo_with_sample_image(self):
        """Demonstra com imagem de exemplo"""
        
        # Criar imagem de exemplo
        sample_image = self.system.create_sample_image()
        
        # Armazenar imagem
        st.session_state['current_image'] = sample_image
        
        # Analisar
        with st.spinner('Analisando imagem de exemplo...'):
            results = self.system.analyze_image(sample_image)
            st.session_state['analysis_results'] = results
        
        st.success("Análise da imagem de exemplo concluída!")
    
    def _display_results(self, results):
        """Exibe os resultados da análise"""
        
        st.subheader("📊 Resultados da Análise")
        
        # Métricas gerais
        col1, col2, col3, col4 = st.columns(4)
        
        with col1:
            st.metric(
                "Classificação",
                f"Classe {results['analysis']['classification']['predicted_class']}",
                f"{results['analysis']['classification']['confidence']:.2f}"
            )
        
        with col2:
            st.metric(
                "Objetos Detectados",
                results['analysis']['detection']['num_objects'],
                "objetos"
            )
        
        with col3:
            st.metric(
                "Segmentos",
                results['analysis']['segmentation']['num_segments'],
                "segmentos"
            )
        
        with col4:
            st.metric(
                "Regiões OCR",
                results['analysis']['ocr']['num_regions'],
                "regiões"
            )
        
        # Tabs para diferentes análises
        tab1, tab2, tab3, tab4, tab5 = st.tabs([
            "🎯 Classificação",
            "🔍 Detecção",
            "✂️ Segmentação",
            "📝 OCR",
            "🔗 Multimodal"
        ])
        
        with tab1:
            self._display_classification_results(results['analysis']['classification'])
        
        with tab2:
            self._display_detection_results(results['analysis']['detection'])
        
        with tab3:
            self._display_segmentation_results(results['analysis']['segmentation'])
        
        with tab4:
            self._display_ocr_results(results['analysis']['ocr'])
        
        with tab5:
            self._display_multimodal_results(results['analysis']['multimodal'])
    
    def _display_classification_results(self, classification):
        """Exibe resultados de classificação"""
        
        # Gráfico de barras das probabilidades
        fig = px.bar(
            x=list(range(len(classification['probabilities']))),
            y=classification['probabilities'],
            title="Probabilidades de Classificação",
            labels={'x': 'Classe', 'y': 'Probabilidade'}
        )
        st.plotly_chart(fig, use_container_width=True)
        
        # Informações detalhadas
        st.write(f"**Classe Predita:** {classification['predicted_class']}")
        st.write(f"**Confiança:** {classification['confidence']:.3f}")
    
    def _display_detection_results(self, detection):
        """Exibe resultados de detecção"""
        
        # Tabela de objetos
        if detection['objects']:
            df = pd.DataFrame(detection['objects'])
            st.dataframe(df, use_container_width=True)
        
        # Gráfico de confiança
        if detection['objects']:
            confidences = [obj['confidence'] for obj in detection['objects']]
            classes = [obj['class'] for obj in detection['objects']]
            
            fig = px.bar(
                x=classes,
                y=confidences,
                title="Confiança por Classe",
                labels={'x': 'Classe', 'y': 'Confiança'}
            )
            st.plotly_chart(fig, use_container_width=True)
    
    def _display_segmentation_results(self, segmentation):
        """Exibe resultados de segmentação"""
        
        st.write(f"**Número de Segmentos:** {segmentation['num_segments']}")
        
        # Visualizar mapa de segmentação
        seg_map = np.array(segmentation['segmentation_map'])
        
        fig = px.imshow(
            seg_map,
            title="Mapa de Segmentação",
            color_continuous_scale='viridis'
        )
        st.plotly_chart(fig, use_container_width=True)
    
    def _display_ocr_results(self, ocr):
        """Exibe resultados de OCR"""
        
        st.write(f"**Texto Extraído:** {ocr['text']}")
        st.write(f"**Número de Regiões:** {ocr['num_regions']}")
        
        # Tabela de regiões
        if ocr['regions']:
            df = pd.DataFrame(ocr['regions'], columns=['X', 'Y', 'Width', 'Height'])
            st.dataframe(df, use_container_width=True)
    
    def _display_multimodal_results(self, multimodal):
        """Exibe resultados multimodais"""
        
        # Métricas
        col1, col2 = st.columns(2)
        
        with col1:
            st.metric(
                "Similaridade",
                f"{multimodal['similarity']:.3f}"
            )
        
        with col2:
            st.metric(
                "Match Semântico",
                "Sim" if multimodal['semantic_match'] else "Não"
            )
        
        # Lista de objetos e entidades
        col1, col2 = st.columns(2)
        
        with col1:
            st.write("**Objetos Detectados:**")
            for obj in multimodal['image_objects']:
                st.write(f"- {obj}")
        
        with col2:
            st.write("**Entidades de Texto:**")
            for entity in multimodal['text_entities']:
                st.write(f"- {entity}")
    
    def _display_system_stats(self):
        """Exibe estatísticas do sistema"""
        
        st.subheader("📈 Estatísticas do Sistema")
        
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.metric("Módulos Ativos", "5", "módulos")
        
        with col2:
            st.metric("Análises Realizadas", "1", "análise")
        
        with col3:
            st.metric("Tempo Médio", "< 1s", "por análise")
    
    def run_app(self):
        """Executa a aplicação"""
        
        self.create_dashboard()

def create_streamlit_app():
    """Cria a aplicação Streamlit"""
    
    print("=== Criando Interface Web com Streamlit ===")
    print("\nPara executar a aplicação, use o comando:")
    print("streamlit run app.py")
    
    # Criar arquivo da aplicação
    app_code = '''
import streamlit as st
import numpy as np
import cv2
import matplotlib.pyplot as plt
from multimodal_system import MultimodalImageAnalysisSystem

def main():
    st.set_page_config(
        page_title="Sistema de Análise de Imagens Multimodal",
        page_icon="🔍",
        layout="wide"
    )
    
    st.title("🔍 Sistema de Análise de Imagens Multimodal")
    
    # Inicializar sistema
    if 'system' not in st.session_state:
        st.session_state.system = MultimodalImageAnalysisSystem()
    
    # Upload de imagem
    uploaded_file = st.file_uploader(
        "Faça upload de uma imagem",
        type=['png', 'jpg', 'jpeg']
    )
    
    if uploaded_file is not None:
        # Processar imagem
        image = np.array(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Exibir imagem
        st.image(image, caption="Imagem enviada", use_column_width=True)
        
        # Analisar
        if st.button("Analisar Imagem"):
            with st.spinner('Analisando...'):
                results = st.session_state.system.analyze_image(image)
            
            # Exibir resultados
            st.success("Análise concluída!")
            
            # Métricas
            col1, col2, col3, col4 = st.columns(4)
            
            with col1:
                st.metric(
                    "Classificação",
                    f"Classe {results['analysis']['classification']['predicted_class']}",
                    f"{results['analysis']['classification']['confidence']:.2f}"
                )
            
            with col2:
                st.metric(
                    "Objetos",
                    results['analysis']['detection']['num_objects']
                )
            
            with col3:
                st.metric(
                    "Segmentos",
                    results['analysis']['segmentation']['num_segments']
                )
            
            with col4:
                st.metric(
                    "Regiões OCR",
                    results['analysis']['ocr']['num_regions']
                )
    
    # Demo com imagem de exemplo
    if st.button("Usar Imagem de Exemplo"):
        sample_image = st.session_state.system.create_sample_image()
        st.image(sample_image, caption="Imagem de exemplo", use_column_width=True)
        
        with st.spinner('Analisando imagem de exemplo...'):
            results = st.session_state.system.analyze_image(sample_image)
        
        st.success("Análise da imagem de exemplo concluída!")
        
        # Exibir resultados
        st.json(results)

if __name__ == "__main__":
    main()
    '''
    
    with open('app.py', 'w') as f:
        f.write(app_code)
    
    print("\nArquivo 'app.py' criado com sucesso!")
    print("\nPara executar a aplicação:")
    print("1. Instale o Streamlit: pip install streamlit")
    print("2. Execute: streamlit run app.py")
    print("3. Acesse: http://localhost:8501")
    
    return app_code

# Criar aplicação
app_code = create_streamlit_app()

### Interface Web Implementada**Funcionalidades da Interface:**1. **Upload de Imagens**: Interface para envio de imagens2. **Análise Interativa**: Botões para executar análises3. **Visualização**: Resultados em tempo real4. **Dashboard**: Métricas e gráficos interativos5. **Relatórios**: Exportação de resultados![Interface Web](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo10/interface_web.png)**Tecnologias Utilizadas:**- **Streamlit**: Framework web- **Plotly**: Gráficos interativos- **Pandas**: Manipulação de dados- **OpenCV**: Processamento de imagem**Referências:**- [Streamlit Documentation](https://docs.streamlit.io/)

## 10.4 Casos de Uso e Aplicações### Aplicações Práticas**1. E-commerce:**- **Busca visual**: Busca por produtos usando imagens- **Categorização**: Categorização automática de produtos- **Moderação**: Moderação de conteúdo visual- **Recomendações**: Recomendações baseadas em visual**2. Saúde:**- **Diagnóstico**: Assistência em diagnóstico médico- **Análise**: Análise de imagens médicas- **Documentação**: Documentação automática- **Educação**: Educação médica**3. Educação:**- **Acessibilidade**: Acessibilidade visual- **Tutoria**: Tutoria multimodal- **Conteúdo**: Criação de conteúdo educacional- **Avaliação**: Avaliação automática**4. Entretenimento:**- **Geração**: Geração de conteúdo visual- **Edição**: Edição automática- **Personalização**: Personalização de conteúdo- **Interação**: Interação multimodal![Casos de Uso](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo10/casos_uso.png)### Vantagens do Sistema**1. Integração:**- **Múltiplas técnicas**: Combinação de diferentes abordagens- **Pipeline completo**: Fluxo de análise end-to-end- **Flexibilidade**: Adaptável a diferentes casos de uso- **Escalabilidade**: Arquitetura modular**2. Performance:**- **Eficiência**: Otimização para diferentes tarefas- **Precisão**: Alta precisão em múltiplas modalidades- **Velocidade**: Processamento em tempo real- **Robustez**: Funciona em diferentes condições**3. Usabilidade:**- **Interface intuitiva**: Fácil de usar- **Visualização clara**: Resultados interpretáveis- **Relatórios**: Documentação automática- **Integração**: Fácil integração em sistemas existentes### Limitações e Desafios**1. Técnicas:**- **Recursos computacionais**: Requer recursos significativos- **Dados**: Depende de dados de qualidade- **Complexidade**: Arquitetura complexa- **Manutenção**: Requer manutenção contínua**2. Práticas:**- **Deploy**: Desafios de deploy em produção- **Escalabilidade**: Escalabilidade horizontal- **Monitoramento**: Monitoramento de performance- **Atualização**: Atualização de modelos**Referências:**- [Deep Learning for Computer Vision - Goodfellow et al.](https://www.deeplearningbook.org/)

## Resumo do Módulo 10

### Principais Conceitos Abordados

1. **Sistema Completo**
   - Arquitetura modular
   - Integração de técnicas
   - Pipeline de análise
   - Visualização de resultados

2. **Implementação Prática**
   - Classificação com CNN
   - Detecção com YOLO
   - Segmentação com U-Net
   - OCR com Tesseract
   - Análise multimodal

3. **Interface Web**
   - Dashboard interativo
   - Upload de imagens
   - Visualização em tempo real
   - Relatórios automáticos

4. **Aplicações Práticas**
   - Casos de uso reais
   - Vantagens e limitações
   - Desafios de implementação
   - Futuro da tecnologia

### Demonstrações Práticas

**1. Sistema Completo:**
   - Análise multimodal
   - Visualização integrada
   - Relatórios automáticos
   - Métricas de performance

**2. Interface Web:**
   - Dashboard Streamlit
   - Upload interativo
   - Visualização em tempo real
   - Exportação de resultados

### Conclusão do Curso

**Conceitos Aprendidos:**
- **Fundamentos**: Processamento digital de imagem
- **Deep Learning**: CNNs e arquiteturas modernas
- **Técnicas Avançadas**: Transfer learning, GANs, VAEs
- **Transformers**: Vision Transformers e atenção
- **Foundation Models**: CLIP, DALL-E, GPT-4V
- **Aplicações**: Casos de uso práticos

**Habilidades Desenvolvidas:**
- **Implementação**: Código prático e funcional
- **Análise**: Interpretação de resultados
- **Integração**: Combinação de técnicas
- **Visualização**: Apresentação de dados
- **Deploy**: Implementação em produção

### Próximos Passos

**Para Continuar Aprendendo:**
1. **Projetos Práticos**: Implementar projetos reais
2. **Datasets**: Trabalhar com datasets públicos
3. **Competições**: Participar de competições
4. **Pesquisa**: Acompanhar papers recentes
5. **Comunidade**: Participar da comunidade

### Referências Principais

- [Deep Learning for Computer Vision - Goodfellow et al.](https://www.deeplearningbook.org/)
- [Computer Vision: Algorithms and Applications - Szeliski](https://szeliski.org/Book/)
- [Attention Is All You Need - Vaswani et al.](https://arxiv.org/abs/1706.03762)

---

**🎉 Parabéns! Você concluiu o curso de Visão Computacional!**

**Obrigado por acompanhar este curso completo sobre Visão Computacional e Deep Learning!**